In [3]:
import tensorflow as tf
import numpy as np
import argparse
import facenet
import os
import sys
import math
import pickle
from scipy import misc
import copy
from align.detect_face import *
from sklearn.externals import joblib

/home/fuxi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
#face detection parameters
minsize = 20 # minimum size of face
threshold = [ 0.6, 0.7, 0.7 ]  # three steps's threshold
factor = 0.709 # scale factor
frame_interval=3 # frame intervals

In [5]:
def to_rgb(img):
  w, h = img.shape
  ret = np.empty((w, h, 3), dtype=np.uint8)
  ret[:, :, 0] = ret[:, :, 1] = ret[:, :, 2] = img
  return ret

In [6]:
print('Creating networks and loading parameters')
gpu_memory_fraction=1.0
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_memory_fraction)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = create_mtcnn(sess, None)

Creating networks and loading parameters


In [7]:
def face_model(model_path,data):
    with tf.Graph().as_default():
        with tf.Session() as sess:
            # Load the model
            facenet.load_model(model_path)
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            feed_dict = { images_placeholder:data, phase_train_placeholder:False }
            emb = sess.run(embeddings, feed_dict=feed_dict)           
    return emb 

In [8]:
#restore pre-trained knn classifier
model=joblib.load('/home/fuxi/face-recognition/20180910-model/knn_classifier.model')
#restore face_model
model_path='/home/fuxi/face-recognition/20180402-114759/20180402-114759.pb'

/home/fuxi/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.18.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [22]:
#obtaining frames from camera--->converting to gray--->converting to rgb
#--->detecting faces---->croping faces--->embedding--->classifying--->print
cv2.namedWindow("Video")#命名一个窗口
video_capture = cv2.VideoCapture(0)  #0号摄像头
c=0

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()  
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #print(frame.shape)  
    timeF = frame_interval
    if((c%timeF == 0) and (ret==True)): #frame_interval==3, face detection every 3 frames    
        find_results=[]
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)       
        if gray.ndim == 2:
            img = to_rgb(gray)  
        bounding_boxes, _ = detect_face(img, minsize, pnet, rnet, onet, threshold, factor)             
        nrof_faces = bounding_boxes.shape[0]#number of faces
        print('找到人脸数目为：{}'.format(nrof_faces))      
        for face_position in bounding_boxes:       
            face_position=face_position.astype(int)  
            #print((int(face_position[0]), int( face_position[1])))
            #word_position.append((int(face_position[0]), int( face_position[1])))
            cv2.rectangle(frame, (face_position[0], face_position[1]),(face_position[2], face_position[3]),(0, 255, 0), 2)
            crop=img[face_position[1]:face_position[3],face_position[0]:face_position[2],]
            crop = cv2.resize(crop, (160, 160), interpolation=cv2.INTER_CUBIC )
            data=crop.reshape(-1,160,160,3)
            #print(face_model(model_path,data).shape)
            emb_data= face_model(model_path,data)
            #emb_data = sess.run([embeddings],feed_dict={images_placeholder: np.array(data),phase_train_placeholder: False })[0]
            predict = model.predict(emb_data) 
            if predict==1:
                find_results.append('me')
            elif predict==2:
                find_results.append('others')       
        cv2.putText(frame,'detected:{}'.format(find_results), (50,100), 
                cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (255, 0 ,0), 
                thickness = 2, lineType = 2)         
    #print(faces)
    c+=1
    # Draw a rectangle around the faces
    # Display the resulting frame
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filename: /home/fuxi/face-recognition/20180402-114759/20180402-114759.pb
找到人脸数目为：1
Model filen

KeyboardInterrupt: 